In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from easydict import EasyDict
import time

In [3]:
URL_PATH = 'https://www.musinsa.com/app/styles/lists'
#options = webdriver.ChromeOptions()

In [ ]:
options = webdriver.ChromeOptions()

# 브라우저를 띄우지 않고 백그라운드에서 크롤링하기 위한 옵션
# options.add_argument('headless') 
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")

In [22]:
from selenium.webdriver.support.select import Select

In [47]:
driver = webdriver.Chrome('./chromedriver',options = options)
driver.get(URL_PATH)
driver.implicitly_wait(2) #페이지를 로딩하는 시간동안 대기

features = EasyDict()

button = driver.find_element(by=By.CSS_SELECTOR, value='button.global-filter__button--mensinsa')
button.click()

container = driver.find_elements_by_css_selector('ul.style-list > li')
COORDI_BASE_PATH = "https://www.musinsa.com/app/styles/views/"
ITEM_BASE_PATH = "https://www.musinsa.com/app/goods/"

codi = []
for ct in container:
    img_src = ct.find_element(By.CSS_SELECTOR, 'div.style-list-item__thumbnail > a').get_attribute('onclick')
    codi_id = img_src.split("'")[1]
    codi_url = COORDI_BASE_PATH + codi_id
    codi.append((codi_id, codi_url))

for codi_id, codi_url in codi:
    driver.get(codi_url)
    driver.implicitly_wait(0.5) #페이지를 로딩하는 시간동안 대기
    
    item_list = driver.find_elements(By.CSS_SELECTOR, 'div.styling_list > div.swiper-slide')
    item_urls = []
    for item in item_list:
        item_url = driver.find_element(By.CSS_SELECTOR, "a.brand_item").get_attribute('href')
        item_urls.append(item_url)
    
    for item_url in item_urls:
        driver.get(item_url)
        driver.implicitly_wait(0.5) #페이지를 로딩하는 시간동안 대기
        
        id = item_url.split('/')[-2]
        name = driver.find_element(By.CSS_SELECTOR, "span.product_title > em").text
        category = driver.find_elements(By.CSS_SELECTOR, "p.item_categories > a")
        big_class = category[0].text
        mid_class = category[1].text
        
        color = []
        colors = driver.find_elements(By.CSS_SELECTOR, "div#goods_opt_area > select#option1 > option")
        for i in range(1, len(colors)):
            color.append(colors[i].text)
            
        size = []
        sizes = driver.find_elements(By.CSS_SELECTOR, "div#goods_opt_area > select#option2 > option")
        for i in range(1, len(sizes)):
            size.append(sizes[i].text)
        
        product_info = driver.find_elements(By.CSS_SELECTOR, "ul.product_article > li > p.product_article_contents")
        brand = product_info[0].find_element(By.CSS_SELECTOR, "strong > a").text
        serial_number = product_info[0].find_element(By.CSS_SELECTOR, "strong").get_attribute('innerHTML').split()[-1]
        season = product_info[1].find_element(By.CSS_SELECTOR, "strong").text
        gender = product_info[1].find_element(By.CSS_SELECTOR, "span.txt_gender").text
        view = product_info[2].find_element(By.CSS_SELECTOR, "strong#pageview_1m").text
        
        price = driver.find_element(By.CSS_SELECTOR, "del.price-del").text
        img_url = driver.find_element(By.CSS_SELECTOR, "div.product-img > img").get_attribute('src')
        
        rating = float(driver.find_element(By.CSS_SELECTOR, value="div.estimate-point > p span").text)
        likes = int(driver.find_element(By.CSS_SELECTOR, value="li.product_section_like span").text)
        try:
            cum_sale = int(driver.find_element(By.CSS_SELECTOR, value="li#li_sales_1y strong").text)
        except :
            cum_sale = None

        buy_age_raw = driver.find_elements(By.CSS_SELECTOR, value="ul.bar_wrap > li span.bar_num")
        buy_age_list = list()
        for buy_age in buy_age_raw :
            buy_age_list.append(int(buy_age.text[:-1]))

        buy_gender_raw = driver.find_elements(By.CSS_SELECTOR, value="dl.label_info > dd")
        buy_gender_list = list()
        for buy_gender in buy_gender_raw :
            buy_gender_list.append(int(buy_gender.text[:-1]))

        buy_age_list = buy_age_list        
        buy_gender_list = buy_gender_list

        guide_all = driver.find_elements(By.CSS_SELECTOR, value="table.table-simple tr")

        four_season_list = list()
        fit_list = list()
        for guide in guide_all :
            t = guide.find_element(By.CSS_SELECTOR, value="th").text
            if t == "계절" :
                seasons = guide.find_elements(By.CSS_SELECTOR, value="td.active")
                for s in seasons :
                    four_season_list.append(s.text)
            elif t == "핏" :
                fits = guide.find_elements(By.CSS_SELECTOR, value="td.active")
                for fit in fits :
                    fit_list.append(fit.text)

        if four_season_list :
            four_season = four_season_list
        else : 
            four_season = None
        
        if fit_list :
            fit = fit_list
        else : 
            fit = None

        
        tags_list = list()
        tags_raw = driver.find_elements(By.CSS_SELECTOR, value="li.article-tag-list a.listItem")
        for tag in tags_raw :
            tags_list.append(tag.text[1:])
        tag = tags_list
        
        
        print(f'codi_id: {codi_id}')
        print(f'id: {id}')
        print(f'name: {name}')
        print(f'brand: {brand}')
        print(f'serial number: {serial_number}')
        print(f'big class: {big_class}')
        print(f'mid class: {mid_class}')
        print(f'color: {color}')
        print(f'gender: {gender}')
        print(f'four season: {four_season}')
        print(f'size: {size}')
        print(f'fit: {fit}')
        print(f'price: {price}')
        print(f'tag: {tag}')
        print(f'view: {view}')
        print(f'cum_sale: {cum_sale}')
        print(f'likes: {likes}')
        print(f'rating: {rating}')
        print(f'season: {season}')
        print(f'buy age: {buy_age_list}')
        print(f'buy gender: {buy_gender_list}')
        print(f'img_url: {img_url}')
        print(f'codi_url: {codi_url}')
        driver.get(codi_url)
        break
    break
driver.close()

codi_id: 26846
id: 2464503
name: 퍼피 플라워 아트웍 반팔티 라벤더
brand: EPDEVELOP
serial number: 503446565
big class: 상의
mid class: 반소매 티셔츠
color: ['RAVENDER']
gender: 남
four season: None
size: ['M', 'L', 'XL']
fit: None
price: 41,000원
tag: ['반팔', '오버핏', '반팔티셔츠', '그래픽', '오버핏반팔', '파스텔', '로고티셔츠']
view: 900회 이상
cum_sale: None
likes: 54
rating: 3.5
season: 2022 S/S
buy age: [23, 11, 14, 33, 4, 15]
buy gender: [62, 38]
img_url: https://image.msscdn.net/images/goods_img/20220404/2464503/2464503_1_500.jpg?t=20220404151808
codi_url: https://www.musinsa.com/app/styles/views/26846
